In [1]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install tqdm
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 8.8 MB/s 
     |████████████████████████████████| 7.6 MB 13.9 MB/s 
     |████████████████████████████████| 182 kB 56.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.5 MB/s 
     |████████████████████████████████| 212 kB 66.8 MB/s 
     |████████████████████████████████| 115 kB 73.6 MB/s 
     |████████████████████████████████| 127 kB 72.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simp

In [2]:
import os
import json
import nltk
from nltk.tokenize import sent_tokenize
import matplotlib.pyplot as plt
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATA_PATH = '/content/drive/MyDrive/nlp-project/data'
BILLSUM_DATA_PATH = os.path.join(DATA_PATH, 'billsum_v4_1')

In [5]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [6]:
TRAIN_DATA_FILE = os.path.join(BILLSUM_DATA_PATH, "us_train_data_final_OFFICIAL.jsonl")
TEST_DATA_FILE = os.path.join(BILLSUM_DATA_PATH, "us_test_data_final_OFFICIAL.jsonl")

dataset = load_dataset("json", data_files={
    "train": TRAIN_DATA_FILE,
    "test": TEST_DATA_FILE,
})

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0a257b7a118a3705/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def preprocess_text(row):
    text = row['text']
    text = text.split('\n')
    text = list(map(lambda x: x.strip(), text))
    text = list(filter(lambda x: True if x else False, text))
    text = " ".join(text)
    row['text'] = text
    return row

In [8]:
dataset = dataset.map(preprocess_text)

  0%|          | 0/18949 [00:00<?, ?ex/s]

  0%|          | 0/3269 [00:00<?, ?ex/s]

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [10]:
from datasets import load_metric
from tqdm import tqdm
import evaluate

In [11]:
def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

def evaluate_summaries(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="text", column_summary="summary"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)
    ):
        inputs = tokenizer(article_batch, max_length=600, truncation=True, padding="max_length", return_tensors="pt")
        summaries = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            length_penalty=0.8,
            num_beams=8,
            max_length=80
        )

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
    
    score = metric.compute()
    return score

In [12]:
rouge_metric = evaluate.load("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

score = evaluate_summaries(dataset['test'], rouge_metric, model, tokenizer, batch_size=8)

100%|██████████| 409/409 [35:16<00:00,  5.18s/it]


In [13]:
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"BART"])

,rouge1,rouge2,rougeL,rougeLsum
BART,0.278497,0.133435,0.203736,0.203683


In [ ]:
#       rouge1	rouge2	    rougeL	    rougeLsum
# BART	0.27226	0.131575	0.200475	0.200599